In [1]:
import os
import uuid
import string
import random
import numpy as np
import pandas as pd
import geopandas as geopd

In [2]:
# modificar segun entorno local
os.chdir("/Volumes/hd_mvf_datasets/henry_data")

# no modificar
folder_data = "1_external"
folder_pipeline = "2_pipeline"
folder_output = "3_output"

In [3]:
yelp_pa_javi1 = pd.read_csv(os.path.join(folder_pipeline, 'Philly_top20.csv'))
yelp_pa_javi2 = pd.read_csv(os.path.join(folder_pipeline, 'Philly_bottom20.csv'))
yelp_pa_javi = pd.concat([yelp_pa_javi1,yelp_pa_javi2]).rename(
        columns={
            "name":"perfil_nombre"
            ,"address":"perfil_direccion"
            #,"city":"county_name"
            #,"state":"state_usps"
            ,"stars":"perfil_calificacion_yelp"
            ,"categories":"perfil_tipo"
            }
    )
list_javi_business_id = yelp_pa_javi["business_id"].unique()
yelp_pa_javi.info()
yelp_pa_javi.head(1)


<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 0 to 19
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   business_id               40 non-null     object 
 1   perfil_nombre             40 non-null     object 
 2   perfil_direccion          40 non-null     object 
 3   latitude                  40 non-null     float64
 4   longitude                 40 non-null     float64
 5   perfil_calificacion_yelp  40 non-null     float64
 6   review_count              40 non-null     float64
 7   is_open                   40 non-null     int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 2.8+ KB


,business_id,perfil_nombre,perfil_direccion,latitude,longitude,perfil_calificacion_yelp,review_count,is_open
0,1w40T5J6br425rMvpS1zgQ,La Cocina del Cafe,1500 S Broad St,39.931674,-75.168458,5.0,24.0,1


In [4]:
n_size = int(120000)
n_areas = int(n_size/6)
n_atributos = int(n_size/3)
n_tipo = int(n_size/10)
data_random_looker = pd.DataFrame({
    "kpi_1_satisfaccion": np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"kpi_2_popularidad": np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"kpi_3_lealtad": np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"cluster1_area":sum([["clstr_area_a"]*n_areas
                    ,["clstr_area_b"]*n_areas
                    ,["clstr_area_c"]*n_areas
                    ,["clstr_area_d"]*n_areas
                    ,["clstr_area_e"]*n_areas
                    ,["clstr_area_f"]*n_areas
                    #,["clstr_area_g"]*n_areas
                    #,["clstr_area_h"]*n_areas
                    ],[])
    ,"cluster2_atributos":sum([["clstr_atributo_a"]*n_atributos
                    ,["clstr_atributo_b"]*n_atributos
                    ,["clstr_atributo_c"]*n_atributos,
                    #,["clstr_atributo_d"]*n_atributos
                    ],[])
    ,"perfil_calificacion_yelp":np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"perfil_calificacion_gmaps":np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"perfil_nombre":((''.join(random.choice(string.ascii_lowercase) for i in range(10))) for i in range(n_size))
    ,"perfil_direccion":((''.join(random.choice(string.ascii_lowercase) for i in range(10))) for i in range(n_size))
    ,"perfil_tipo":["comida_a","comida_b","comida_c","comida_d","comida_e"]*2*n_tipo
    ,"perfil_otros":((''.join(random.choice(string.ascii_lowercase) for i in range(10))) for i in range(n_size))
    })
data_random_looker["kpi_4_indice"] = (
    (data_random_looker["kpi_1_satisfaccion"]*0.4) +
    (data_random_looker["kpi_2_popularidad"]*0.3) +
    (data_random_looker["kpi_3_lealtad"]*0.3)
    )
data_random_looker.head()

,kpi_1_satisfaccion,kpi_2_popularidad,kpi_3_lealtad,cluster1_area,cluster2_atributos,perfil_calificacion_yelp,perfil_calificacion_gmaps,perfil_nombre,perfil_direccion,perfil_tipo,perfil_otros,kpi_4_indice
0,26.866463,67.066979,63.604401,clstr_area_a,clstr_atributo_a,66.046228,56.616090,owuqcqrfro,yulsxvilpw,comida_a,dckkeaczwv,49.947999
1,53.148845,69.074220,39.867821,clstr_area_a,clstr_atributo_a,78.437944,37.834119,hhqzijjqgt,fxbkebwray,comida_b,qafdibfqdh,53.942150
2,56.772218,28.488466,68.208081,clstr_area_a,clstr_atributo_a,49.341668,62.710865,ogunhdtpvi,vyrvibyqfp,comida_c,cpkvrlktbx,51.717851
3,55.188956,70.507845,21.236658,clstr_area_a,clstr_atributo_a,47.204993,69.639768,wqmsiinliy,gemfwwursz,comida_d,bepqsasrtj,49.598933
4,59.226742,67.413365,56.851741,clstr_area_a,clstr_atributo_a,98.287172,63.287363,yfjvdabdpc,oitvsyvorn,comida_e,bysgeonygp,60.970229


In [5]:
yelp_geodf_zcta_census = pd.read_pickle(
    os.path.join(folder_pipeline, 'data_zcta_varsxarea.pkl')).dropna().iloc[:,[0,1,2,17,18,19]]
yelp_geodf_zcta_census.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 165586 entries, 0 to 166416
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  165586 non-null  string 
 1   latitude     165586 non-null  Float64
 2   longitude    165586 non-null  Float64
 3   state_name   165586 non-null  object 
 4   state_usps   165586 non-null  object 
 5   county_name  165586 non-null  object 
dtypes: Float64(2), object(3), string(1)
memory usage: 9.2+ MB


In [6]:
#yelp_geodf_zcta_census = yelp_geodf_zcta_census[~yelp_geodf_zcta_census.business_id.isin(list_javi_business_id)].copy()
n_sample = len(data_random_looker.index)
yelp_geodf_zcta_census_sample = yelp_geodf_zcta_census.sample(n=n_sample)
yelp_geodf_zcta_census_sample.reset_index(drop=True, inplace=True)
data_random_looker = data_random_looker.join(yelp_geodf_zcta_census_sample)
data_random_looker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   kpi_1_satisfaccion         80000 non-null  float64
 1   kpi_2_popularidad          80000 non-null  float64
 2   kpi_3_lealtad              80000 non-null  float64
 3   cluster1_area              80000 non-null  object 
 4   cluster2_atributos         80000 non-null  object 
 5   perfil_calificacion_yelp   80000 non-null  float64
 6   perfil_calificacion_gmaps  80000 non-null  float64
 7   perfil_nombre              80000 non-null  object 
 8   perfil_direccion           80000 non-null  object 
 9   perfil_tipo                80000 non-null  object 
 10  perfil_otros               80000 non-null  object 
 11  kpi_4_indice               80000 non-null  float64
 12  business_id                80000 non-null  string 
 13  latitude                   80000 non-null  Flo

In [7]:
#yelp_geodf_zcta_census_javi = pd.concat([data_random_looker,yelp_pa_javi])
data_random_looker["geo_looker"] = str(
    str(data_random_looker["latitude"])+","+str(data_random_looker["longitude"]))
data_random_looker.drop(columns=["latitude","longitude"], inplace=True)

In [8]:
data_random_looker.to_csv(
    os.path.join(folder_output, 'data_output_lookerstudio.csv'))
#yelp_geodf_zcta_census_javi.info()
data_random_looker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   kpi_1_satisfaccion         80000 non-null  float64
 1   kpi_2_popularidad          80000 non-null  float64
 2   kpi_3_lealtad              80000 non-null  float64
 3   cluster1_area              80000 non-null  object 
 4   cluster2_atributos         80000 non-null  object 
 5   perfil_calificacion_yelp   80000 non-null  float64
 6   perfil_calificacion_gmaps  80000 non-null  float64
 7   perfil_nombre              80000 non-null  object 
 8   perfil_direccion           80000 non-null  object 
 9   perfil_tipo                80000 non-null  object 
 10  perfil_otros               80000 non-null  object 
 11  kpi_4_indice               80000 non-null  float64
 12  business_id                80000 non-null  string 
 13  state_name                 80000 non-null  obj

In [9]:
list_business_id = []
for i in range(min(data_random_looker.index),max(data_random_looker.index)+1):
    business_id = str(uuid.uuid4().hex)
    dict_case = {'index':i,'business_id':business_id}
    list_business_id.append(dict_case)
data_random_business_id = pd.DataFrame(list_business_id).set_index('index')